Import packages

In [1]:
import tensorflow as tf
import create_dataset
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Define generic functions to visualize data

In [2]:
def showImage(im):
    if (im.shape[2] == 1):
        im = im[:, :, 0]
    plt.imshow(im)
    plt.show()

Define generic functions to initialize weight/bias variables, and to define common convolution and pooling operations

In [3]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name='weights')

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name='biases')

In [4]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

Define the network architecture + loss function and metrics

In [25]:
# Input/Output placeholders
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 2], name='output')

num_conv_layers = 10
num_fc_layers = 10

# Layer 1
with tf.name_scope('1/Conv'):
    W_conv1 = weight_variable([3, 3, 1, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

i = i - 1
last_conv_layer = h_pool1
# Layer 2
for i in range(0, num_conv_layers):
    with tf.name_scope('{}/Conv'.format(i+2)):
        W_conv2 = weight_variable([3, 3, 32, 32])
        b_conv2 = bias_variable([32])
        h_conv2 = tf.nn.relu(conv2d(last_conv_layer, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        last_conv_layer = h_pool2


layers = [tf.reshape(last_conv_layer, [-1, 7*7*32])]
keep_prob = tf.placeholder(tf.float32)
prev_layer_size = 7*7*32                     
# Create FC layers
for i in range(0, num_fc_layers):
    with tf.name_scope('{}/FC'.format(i)):
        W_fc = weight_variable([prev_layer_size, 1024])
        b_fc = bias_variable([1024])
        h_fc = tf.nn.relu(tf.matmul(layers[-1], W_fc) + b_fc)
        h_fc_drop = tf.nn.dropout(h_fc, keep_prob)
        layers.append(h_fc_drop)
        prev_layer_size = 1024

# Readout
with tf.name_scope('5/Readout'):
    W_fc2 = weight_variable([1024, 1])
    b_fc2 = bias_variable([1])
    y_conv = tf.matmul(layers[-1], W_fc2) + b_fc2
    
# Loss
with tf.name_scope('6/Loss'):
    y_sincos = [tf.sin(y_conv), tf.cos(y_conv)]

# Define loss
l2_loss = tf.reduce_mean(
    tf.nn.l2_loss(tf.subtract(y_sincos, y_)))
train_step = tf.train.AdamOptimizer(1e-4).minimize(l2_loss)

ValueError: Dimensions must be equal, but are 3136 and 1568 for '0/FC_9/MatMul' (op: 'MatMul') with input shapes: [?,3136], [1568,1024].

Train the network per-batch

In [23]:
batch_size = 40
logging_frequency = 5
iterations = 50

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(iterations):
    batch = create_dataset.generate_data(batch_size)
    if i % logging_frequency == 0:
      loss = sess.run(l2_loss, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}) 
      print('step %d, training loss %g' % (i, loss/batch_size))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
    
  visualization_size = 5
  vis_batch = create_dataset.generate_data(visualization_size)
  output = sess.run(y_conv, feed_dict={x: vis_batch[0], y_: vis_batch[1], keep_prob: 1.0})
  for i in range(visualization_size):
    showImage(vis_batch[0][i])
    print ("True angle: " + str(vis_batch[2][i][0]))
    print ("True sin/cos: " + str(vis_batch[1][i]))
    print ("Predicted angle: " + str((output[i]*180/math.pi)%360))

InvalidArgumentError: Input to reshape is a tensor with 1280 values, but the requested shape requires a multiple of 1568
	 [[Node: Reshape_7 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](11/Conv_2/MaxPool, Reshape_7/shape)]]

Caused by op u'Reshape_7', defined at:
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop